In [ ]:
import wandb
api = wandb.Api()

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
summary_list = [] 
config_list = [] 
name_list = [] 
base_config_list = []

sweep_ids = "klz1a5oy", "z5iljqca"
base_configs = ['tl_fast1', 'tl_best5']
for i, sweep_id in enumerate(sweep_ids):
    sweep = api.sweep(f"ogait/thesis_gravity_spy/{sweep_id}")
    base_config = base_configs[i]
    for run in sweep.runs:
        summary = {k:v for k,v in run.summary.items() if k in ('f1_score', 'combined_f1_time')}
        summary_list.append(summary) 

        config = {k:v for k,v in run.config.items() if k in ('tfm_shift_fraction', 'tfm_zoom_range')}
        config_list.append(config) 

        base_config_list.append(base_config)
        name_list.append(run.name)       

summary_df = pd.DataFrame.from_records(summary_list) 
config_df = pd.DataFrame.from_records(config_list) 
name_df = pd.DataFrame({'name': name_list})
base_config_df = pd.DataFrame({'base_config': base_config_list})
all_df = pd.concat([name_df, base_config_df, config_df,summary_df], axis=1)

In [ ]:
all_df.tail()

,name,base_config,tfm_zoom_range,tfm_shift_fraction,f1_score,combined_f1_time
55,lunar-sweep-5,tl_best5,0.070,0.050,0.978641,0.975689
56,crimson-sweep-4,tl_best5,0.045,0.035,0.977684,0.974748
57,quiet-sweep-3,tl_best5,0.080,0.095,0.971780,0.968846
58,curious-sweep-2,tl_best5,0.015,0.065,0.964341,0.961425
59,icy-sweep-1,tl_best5,0.085,0.070,0.982225,0.979287


In [ ]:
fast1_top = all_df.loc[all_df['base_config'] == 'tl_fast1'].sort_values('f1_score', ascending=False)[:3]

In [ ]:
best5_top = all_df.loc[all_df['base_config'] == 'tl_best5'].sort_values('f1_score', ascending=False)[:3]

In [ ]:
top_runs = fast1_top.merge(best5_top, how='outer')
configs = ['tl_fast1_aug1', 'tl_fast1_aug2', 'tl_fast1_aug3', 'tl_best5_aug1', 'tl_best5_aug2', 'tl_best5_aug3']
top_runs['configuration'] = configs
top_runs.rename(columns = {
                            'base_config': 'base configuration', 
                            'tfm_zoom_range': 'zoom_range', 
                            'tfm_shift_fraction': 'shift_fraction'
                          }, inplace=True)
top_runs

,name,base configuration,zoom_range,shift_fraction,f1_score,combined_f1_time,configuration
0,different-sweep-10,tl_fast1,0.000,0.085,0.984340,0.982596,tl_fast1_aug1
1,jolly-sweep-22,tl_fast1,0.015,0.085,0.984201,0.982510,tl_fast1_aug2
2,revived-sweep-26,tl_fast1,0.035,0.005,0.982573,0.980752,tl_fast1_aug3
3,icy-sweep-1,tl_best5,0.085,0.070,0.982225,0.979287,tl_best5_aug1
4,vital-sweep-10,tl_best5,0.015,0.005,0.980001,0.977088,tl_best5_aug2
5,lunar-sweep-5,tl_best5,0.070,0.050,0.978641,0.975689,tl_best5_aug3


In [ ]:
with open('../figs/glitches/13_optimized_transfer_aug_table.tex', 'w') as f:
    top_runs[['base configuration', 'zoom_range', 'shift_fraction', 'f1_score', 'configuration']].to_latex(f, float_format="{:0.4f}".format, index=False)